In [783]:
from matplotlib import pyplot as plt
from pathlib import Path 
from PIL import Image
from scipy import ndimage, misc
import numpy as np
import cv2
import os
import math

In [784]:
def remove_noise(imgray,row,col):
    ori_imgray = imgray.copy()
    resolution = 0
    resolution = 4

    

    invert_color(imgray, row, col)
    ret,thresh = cv2.threshold(imgray,10,255,0)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    

    while (len(contours)>1):
        print("resolution",resolution)
        resolution += 1
        imgray = ori_imgray.copy()
        

        resolution_image(row,col,resolution,imgray)
        
        invert_color(imgray, row, col)
        ret,thresh = cv2.threshold(imgray,10,255,0)
        im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
        #get the coordinate from land
        cnt = contours[0]
        #print ('contours',contours)
        
        #land_cor=[]
        #for k in range(len(cnt)):
            
        #    land_cor.append(cnt[k])
            
        #print ('land_cor',land_cor)
       
        
        land_cor=[]
        for i in range(len(cnt)):
            tmp = cnt[i]
            for j in range(len(tmp)):
                land_cor.append(  (tmp[j][0], tmp[j][1]) )
    
    
    
    test_output = os.path.join('./tochi_test37/'+'removenoisefun'+'.png' )
    print("output_path is", test_output)
    cv2.imwrite(test_output, imgray)

    
    #ivert image color
    invert_color(imgray, row, col)

    return imgray, land_cor, contours
 

In [785]:
def resolution_image(row,col,resolution,imgray):
    for r in range (resolution):
        wpts = []
        for i in range(1,row-1):
            for j in range(1,col-1):
                if (imgray[i][j]==255):
                    wpts.append((i, j))
              
        for k in range(len(wpts)):
            i, j = wpts[k]
            #imgray[i][j]=255
            imgray[i+1][j]=255
            imgray[i+1][j+1]=255
            imgray[i][j+1]=255
            imgray[i-1][j+1]=255
            imgray[i-1][j]=255
            imgray[i-1][j-1]=255
            imgray[i][j-1]=255
            imgray[i+1][j-1]=255
            
            
    for a in range (resolution):
        wpts=[]

        for i in range(1,row-1):
            for j in range(1,col-1):
                if (imgray[i][j]!=255):
                    wpts.append((i, j))
    
        for k in range(len(wpts)):
            i, j = wpts[k]
            imgray[i+1][j]=imgray[i][j]
            imgray[i+1][j+1]=imgray[i][j]
            imgray[i][j+1]=imgray[i][j]
            imgray[i-1][j+1]=imgray[i][j]
            imgray[i-1][j]=imgray[i][j]
            imgray[i-1][j-1]=imgray[i][j]
            imgray[i][j-1]=imgray[i][j]
            imgray[i+1][j-1]=imgray[i][j]

In [786]:
def road_detection(new_imgray,ori_imgray,row,col):
    
    for i in range(row):
        for j in range(col):
            if (new_imgray[i][j]!=255):
                ori_imgray[i][j]=255
 
    return ori_imgray
  

In [787]:
def hough_linedetection(ori_imgray):
 
    backtorgb = cv2.cvtColor(ori_imgray,cv2.COLOR_GRAY2RGB)
    edges = cv2.Canny(backtorgb,100,200,apertureSize = 3)
    minLineLength = 50
    maxLineGap = 10
    threshold = 15
 
    lines = cv2.HoughLinesP(edges,1,np.pi/180,threshold,minLineLength,maxLineGap)
  
    return lines


In [788]:
def median_filter(land_cor):
    median_result = ndimage.median_filter(imgray, size=(3, 3))
    
    return median_result

In [789]:
def getPoints(x1, y1, x2, y2, gp_lst):

    if (x1==x2):
        for y in range (y1,y2+1):
                gp_lst.append((x1,y))
    else:
    
        m = (y2-y1) / (x2-x1)
        b = y1 - m * x1
        for x in range(x1, x2+1):
            y = m * x + b
            y=int(y)
            gp_lst.append((x,y))
    #print (gp_lst)
    return gp_lst

In [790]:
def line_dist(x1,y1,x2,y2):
    x=abs(x1-x2)
    y=abs(y1-y2)
    sum_line=math.sqrt((x**2)+(y**2))
    return sum_line

In [791]:
def getMinLengthLineSegment(point_lst):

    min_point=[]
    min_length = line_dist(point_lst[0][0], point_lst[0][1], point_lst[1][0], point_lst[1][1])
    min_point.append(point_lst[0])
    min_point.append(point_lst[1])
    print("min_point is",min_point)

    for i in range(len(point_lst)):
        if (i != len(point_lst)-1):
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[i+1]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line<min_length):
                min_length=sum_line
                min_point[0]=point_lst[i]
                min_point[1]=point_lst[i+1]
                print("min_point is",min_point)
        else:
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[0]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line<min_length):
                min_length=sum_line
                min_point[0]=point_lst[i]
                min_point[1]=point_lst[0]
                print("min_point is",min_point)
                
        print ("min_legth is",min_length) 
        print ("min_point is",min_point) 
        return min_length, min_point

In [792]:
def getMaxLengthLineSegment(point_lst):

    max_point=[]
    max_length = line_dist(point_lst[0][0], point_lst[0][1], point_lst[1][0], point_lst[1][1])
    max_point.append(point_lst[0])
    max_point.append(point_lst[1])

    for i in range(len(point_lst)):
        if (i != len(point_lst)-1):
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[i+1]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line>max_length):
                max_length=sum_line
                max_point[0]=point_lst[i]
                max_point[1]=point_lst[i+1]

        else:
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[0]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line>max_length):
                max_length=sum_line
                max_point[0]=point_lst[i]
                max_point[1]=point_lst[0]
                
                
    print ("max_legth is",max_length) 
    print ("max_point is",max_point)             
    return max_length, max_point

In [793]:
def Gravity_Center(point_lst):
    count_x=0
    count_y=0
    for i in range(len(point_lst)):
        if (i==0):
            count_x=point_lst[i][0]
            count_y=point_lst[i][1]
        else:
            count_x+=point_lst[i][0]
            count_y+=point_lst[i][1]
            
    GC_x= count_x / len(point_lst)
    GC_y= count_y / len(point_lst)
    GC_x=int(GC_x)
    GC_y=int(GC_y)
    print(GC_x)
    print(GC_y)
    
    return GC_x,GC_y
        #print('point_lst is',point_lst[i])

In [794]:
def dist(x1,y1, x2,y2, x3,y3): # x3,y3 is the point
    px = x2-x1
    py = y2-y1

    something = px*px + py*py

    u =  ((x3 - x1) * px + (y3 - y1) * py) / float(something)

    if u > 1:
        u = 1
    elif u < 0:
        u = 0

    x = x1 + u * px
    y = y1 + u * py

    dx = x - x3
    dy = y - y3

    # Note: If the actual distance does not matter,
    # if you only want to compare what this function
    # returns to other results of this function, you
    # can just return the squared distance instead
    # (i.e. remove the sqrt) to gain a little performance

    dist = math.sqrt(dx*dx + dy*dy)
    print (dist)

    return dist

In [795]:
def invert_color(imgray, row, col):
    for i in range(row):
        for j in range(col):
            imgray[i][j] = 255 - imgray[i][j]

In [796]:
def enhance_image(imgray):
    print('ENHACE')
    #median_result=median_filter(imgray)
    #output_path = os.path.join('./tochi_test37/'+'median_filter'+'.bmp' )
    #cv2.imwrite(output_path, median_result)
    #print('median print finish')

In [797]:
def run(myfile,line):

    im = cv2.imread(myfile)
 
    check_file = Path(myfile) 
    if check_file.is_file():
        print ('file exist')
    else:
        print ('not')

    #Gray tone
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    ori_imgray = imgray.copy()
    ori2_imgray = imgray.copy()   

    #enhance_image(imgray)
    
    row=imgray.shape[0]
    col=imgray.shape[1]
    new_imgray, land_cor, contours = remove_noise(ori_imgray, row, col)
    #for i in range(row):
    #    print (imgray[i][0])
    #imgray, land_cor = remove_noise(imgray, row, col)
    print (row, col)
    
    
    rode_imgray=road_detection(new_imgray,ori2_imgray,row,col)
 
    
    
    #draw the line of contour
    #for i in range(1, len(land_cor)):
    #    x1, y1 = land_cor[i]
    #    x2, y2 = land_cor[i-1]
    #    cv2.line( backtorgb,(x1,y1),(x2,y2),(0,255,0),2   )
    
    
  
    
    min_length, min_point=getMinLengthLineSegment(land_cor)
    max_length, max_point=getMaxLengthLineSegment(land_cor)
    
 
    #hough
    backtorgb = cv2.cvtColor(rode_imgray,cv2.COLOR_GRAY2RGB)
    hough_lines=hough_linedetection(rode_imgray)
    
    for x in range(0, len(hough_lines)):
        for x1,y1,x2,y2 in hough_lines[x]:
            cv2.line(backtorgb,(x1,y1),(x2,y2),(0,255,0),2)
    
  
    hough_output = os.path.join('./tochi_test37/'+line+'_hough'+'.png' )
 
    cv2.imwrite(hough_output, backtorgb)
    ret,thresh = cv2.threshold(new_imgray,127,255,0)
    
    #findContours
    #im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #get all point from hough
    gp_lst=[]
    for i in range (0,len(hough_lines)):
         for x1,y1,x2,y2 in hough_lines[i]:
                getPoints(x1, y1, x2, y2, gp_lst)
                
    
    #gravity of road 
    GC_x, GC_y=Gravity_Center(gp_lst)
    print ("GC_x, GC_y",GC_x, GC_y)
    
    cv2.line(backtorgb,(0,0),(GC_x, GC_y),(0,255,0),2)
    
    gravity_output = os.path.join('./tochi_test37/'+'gravity_output'+'.png' )
    cv2.imwrite(gravity_output, backtorgb)
    
    #point to line distance
    
    x1,y1=max_point[0]
    x2,y2=max_point[1]
    
    #print ('max_point',type(max_point))
    #print ('min_point',type(min_point))
    
    dist(x1,y1, x2,y2, GC_x, GC_y)
    
    x1,y1=min_point[0]
    x2,y2=min_point[1]
    dist(x1,y1, x2,y2, GC_x, GC_y)        
    p
  

In [798]:
output_dir='./tochi'
fp=open('./tochi/image_list.txt','r')
line =  fp.readline()


while line:
   
    line = line.strip()
    
    myfile = './tochi/' + line
    
    print (myfile)
    run(myfile,line)
    line =  fp.readline()


print("out of while")    

    
    

./tochi_test37/C62145A23.png
file exist
resolution 4
output_path is ./tochi_test37/removenoisefun.png
output_path is ./tochi_test37/remoafterinvert.png
output_path is ./tochi_test37/remoafterinvert_ori.png
600 600
i am in road detection
output_path is ./tochi_test37/road_detection_new_imgray.png
i am in road detection
output_path is ./tochi_test37/road_detection_ori_imgray.png
min_point is [(533, 188), (532, 189)]
min_legth is 1.4142135623730951
min_point is [(533, 188), (532, 189)]
max_legth is 15.0
max_point is [(311, 259), (311, 274)]
108
346
GC_x, GC_y 108 346
215.3903433304288
452.133829745132
out of while


In [799]:
print("all program is done")

all program is done
